In [1]:
import pickle
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

#### 파일 불러오기

In [2]:
notes_info = pd.read_csv('../data/all_note_data.csv') # rows : 973
with open('../data/notes.pkl', 'rb') as f:
    notes_dict = pickle.load(f)  # len : 973

#### filling empty category values
1. top_notes, heart_notes, base_notes를 한 리스트
2. note들 번호를 category_idx로 치환
3. count
4. 상위 1뽑기
5. category에 채워넣기

In [3]:
df = pd.read_csv('../data/1st treatment_perfume.csv', index_col=0)
# categories열이 비어있는 row들을 따로 빼서 준비
emp_df = df[df['categories'] == '[]']

# *_notes 들의 값이 str이므로 list로 새로운 열에 저장
from ast import literal_eval
emp_df['top_notes_list'] = emp_df['top_notes'].apply(literal_eval)
emp_df['heart_notes_list'] = emp_df['heart_notes'].apply(literal_eval)
emp_df['base_notes_list'] = emp_df['base_notes'].apply(literal_eval)
emp_df.head()

,pk,model,name,thumbnail,gender,top_notes,heart_notes,base_notes,seasons,availability,brand,categories,price,top_notes_list,heart_notes_list,base_notes_list
4,26120018,perfumes.perfume,1872 for Men,26120018.jpg,1,"[229, 562, 708, 715]",[],[],[],True,682,[],389.00,"[229, 562, 708, 715]",[],[]
15,26120043,perfumes.perfume,Lavanda Tonica,26120043.jpg,0,"[96, 512, 515, 661]",[],[],[],True,61,[],29.98,"[96, 512, 515, 661]",[],[]
28,26120067,perfumes.perfume,Agua de Loewe,26120067.jpg,0,"[96, 971]",[],"[26, 193, 624, 808]",[],True,1813,[],10.50,"[96, 971]",[],"[26, 193, 624, 808]"
35,26120089,perfumes.perfume,Ambre Sultan,26120089.jpg,0,"[26, 920]",[],[],[],True,2767,[],92.17,"[26, 920]",[],[]
41,26120104,perfumes.perfume,Annayaké Pour Lui,26120104.jpg,1,"[888, 908]",[],[],[],True,218,[],59.99,"[888, 908]",[],[]


In [4]:
# 'notes'라는 열을 만들고 top, middle, base note들의 값을 합쳐서 입력
emp_df['notes'] = None
for i in emp_df.index:
    new_top = emp_df['top_notes_list'][i]
    new_middle = emp_df['heart_notes_list'][i]
    new_base = emp_df['base_notes_list'][i]
    
    emp_df['notes'][i] = new_top + new_middle + new_base

emp_df.head()

,pk,model,name,thumbnail,gender,top_notes,heart_notes,base_notes,seasons,availability,brand,categories,price,top_notes_list,heart_notes_list,base_notes_list,notes
4,26120018,perfumes.perfume,1872 for Men,26120018.jpg,1,"[229, 562, 708, 715]",[],[],[],True,682,[],389.00,"[229, 562, 708, 715]",[],[],"[229, 562, 708, 715]"
15,26120043,perfumes.perfume,Lavanda Tonica,26120043.jpg,0,"[96, 512, 515, 661]",[],[],[],True,61,[],29.98,"[96, 512, 515, 661]",[],[],"[96, 512, 515, 661]"
28,26120067,perfumes.perfume,Agua de Loewe,26120067.jpg,0,"[96, 971]",[],"[26, 193, 624, 808]",[],True,1813,[],10.50,"[96, 971]",[],"[26, 193, 624, 808]","[96, 971, 26, 193, 624, 808]"
35,26120089,perfumes.perfume,Ambre Sultan,26120089.jpg,0,"[26, 920]",[],[],[],True,2767,[],92.17,"[26, 920]",[],[],"[26, 920]"
41,26120104,perfumes.perfume,Annayaké Pour Lui,26120104.jpg,1,"[888, 908]",[],[],[],True,218,[],59.99,"[888, 908]",[],[],"[888, 908]"


In [5]:
# 사용이 끝난 *_notes_list들을 삭제
emp_df = emp_df.drop(['top_notes_list', 'heart_notes_list', 'base_notes_list'], axis = 1)
emp_df.columns

Index(['pk', 'model', 'name', 'thumbnail', 'gender', 'top_notes',
       'heart_notes', 'base_notes', 'seasons', 'availability', 'brand',
       'categories', 'price', 'notes'],
      dtype='object')

### note를 category로
* concat_notes : pk & name
* all_note_data : note(name) & category_idx

In [6]:
notes_name = pd.read_csv('../data/concat_notes.csv')
notes_name.head(2)

,pk,model,name,kor_name
0,1,perfumes.note,absinthe,압생트
1,2,perfumes.note,acacia,아카시아


#### emp_df의 notes를 name으로 변환

In [7]:
import itertools
for i in emp_df.index:
    alist = emp_df['notes'][i]
    for j in range(len(alist)):
        alist[j] = notes_name[notes_name['pk'] == alist[j]]['name']
    emp_df['notes'][i] = list(itertools.chain(*emp_df['notes'][i]))
emp_df.head(2)

,pk,model,name,thumbnail,gender,top_notes,heart_notes,base_notes,seasons,availability,brand,categories,price,notes
4,26120018,perfumes.perfume,1872 for Men,26120018.jpg,1,"[229, 562, 708, 715]",[],[],[],True,682,[],389.00,"[clary sage, mandarin orange, pepper, petitgrain]"
15,26120043,perfumes.perfume,Lavanda Tonica,26120043.jpg,0,"[96, 512, 515, 661]",[],[],[],True,61,[],29.98,"[bergamot, lavender, lemon, orange blossom]"


#### name을 categoey_idx로
* notes가 notes_info['note']같을때 --> notes_info['category_idx']

In [8]:
for i in emp_df.index:
    alist = emp_df['notes'][i]
    for j in range(len(alist)):
        alist[j] = notes_info[notes_info['note'] == alist[j]]['category_idx']
    emp_df['notes'][i] = list(itertools.chain(*emp_df['notes'][i]))
emp_df.head(2)

,pk,model,name,thumbnail,gender,top_notes,heart_notes,base_notes,seasons,availability,brand,categories,price,notes
4,26120018,perfumes.perfume,1872 for Men,26120018.jpg,1,"[229, 562, 708, 715]",[],[],[],True,682,[],389.00,"[5, 1, 6, 1]"
15,26120043,perfumes.perfume,Lavanda Tonica,26120043.jpg,0,"[96, 512, 515, 661]",[],[],[],True,61,[],29.98,"[1, 3, 1, 4]"


In [9]:
from collections import Counter
# 'categories'에 전체 notes에서 가장 많이 등장하는 카테고리 인덱스를 입력 (이미 채워져 있던 값들과 같은 포멧으로)
for i in emp_df.index:
    counter_object = Counter(emp_df['notes'][i])
    emp_df['categories'][i] ="[" + str(max(counter_object, key=counter_object.get)) + "]"
# merge를 위해 다 사용한 'notes' 열 삭제
emp_df = emp_df.drop('notes', axis = 1)

In [10]:
emp_df.head(2)

,pk,model,name,thumbnail,gender,top_notes,heart_notes,base_notes,seasons,availability,brand,categories,price
4,26120018,perfumes.perfume,1872 for Men,26120018.jpg,1,"[229, 562, 708, 715]",[],[],[],True,682,[1],389.00
15,26120043,perfumes.perfume,Lavanda Tonica,26120043.jpg,0,"[96, 512, 515, 661]",[],[],[],True,61,[1],29.98


합치기

In [11]:
# 이미 카테고리가 채워져 있는 값들과 새로 채운 값들을 합치기
fill_df = df[df['categories'] != '[]']
concat_df = pd.concat([fill_df, emp_df])
# 원래의 데이터 프레임 형태로 되돌리기 위해 인덱스로 정렬 후 csv파일로 저장
concat_df = concat_df.sort_index()
concat_df.to_csv('../data/cat_filled.csv', index = False)

In [15]:
# 저장된 csv파일이 제대로 생성되었는지 확인
df = pd.read_csv('../data/cat_filled.csv')
df.head()

,pk,model,name,thumbnail,gender,top_notes,heart_notes,base_notes,seasons,availability,brand,categories,price
0,26120003,perfumes.perfume,X-Centric,26120003.jpg,1,"[178, 224, 229, 292, 388, 395, 646]","[358, 545, 785]","[26, 193, 624, 692]","[1, 2, 3, 4]",True,895,[3],21.00
1,26120006,perfumes.perfume,GFT,26120006.jpg,1,"[96, 515, 562]","[292, 512, 886]","[193, 624, 649]",[3],True,1170,"[3, 5, 8]",62.00
2,26120008,perfumes.perfume,Citron Citron,26120008.jpg,0,"[515, 530, 660]","[82, 604]","[178, 193, 649]","[2, 3]",True,2054,[5],99.07
3,26120015,perfumes.perfume,30cc / 50cc / 100cc,26120015.jpg,1,"[96, 388]","[45, 473, 793, 896]","[624, 649]",[3],True,642,[5],22.67
4,26120018,perfumes.perfume,1872 for Men,26120018.jpg,1,"[229, 562, 708, 715]",[],[],[],True,682,[1],389.00


In [16]:
df[df['categories']=='[0]']

,pk,model,name,thumbnail,gender,top_notes,heart_notes,base_notes,seasons,availability,brand,categories,price
